In [ ]:
# Setup autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# Import libraries
import itertools
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import haversine_distances

# Tweak display settings for tables
pd.options.display.max_columns = 999

Read data

In [ ]:
DATA_PATH = "../../data/Business_Licenses.csv"
DTYPE_DICT = {
    'ZIP CODE': str,
    'BUSINESS ACTIVITY ID': str,
    'BUSINESS ACTIVITY': str,
}
DATE_COLS = ['LICENSE TERM EXPIRATION DATE', 'DATE ISSUED']

license_df = pd.read_csv(DATA_PATH,
                 dtype=DTYPE_DICT,
                 parse_dates=DATE_COLS)
license_df.shape

Define `get_locations()` and `reshape_and_make_label()`

In [ ]:
def get_locations(input_df):
    '''
    Takes license-level data and returns a dataframe with location attributes
        for each account-site.
    '''
    # Columns to return
    LOCATION_COLS = ['ACCOUNT NUMBER', 'SITE NUMBER', 'ADDRESS', 'CITY',
                     'STATE', 'ZIP CODE', 'WARD', 'POLICE DISTRICT',
                     'LATITUDE', 'LONGITUDE', 'LOCATION']

    # Drop rows if these columns have NA
    NA_COLS = ['LATITUDE', 'LONGITUDE', 'LOCATION']

    df = input_df.copy(deep=True)[LOCATION_COLS] \
        .dropna(subset=NA_COLS) \
        .drop_duplicates() \
        .sort_values(by=['ACCOUNT NUMBER', 'SITE NUMBER'])

    return df

addresses = get_locations(df)

addresses.head()

In [ ]:
def reshape_and_create_label(input_df):
    '''
    Processes raw business license-level dataframe into account-site-year level
     dataframe. Extracts years from min/max year and expands dataframe into
     account-site-year level.

    Currently hardcoded to require columns for ACCOUNT NUMBER, SITE NUMBER,
     DATE ISSUED, and LICENSE TERM EXPIRATION DATE

    Input:  input_df - license-level dataframe
    Output: result_df - business-year-level df with not_renewed_2yrs label
    '''

    # Aggregate by account-site and get min/max/expiry dates for licenses
    df = input_df.copy(deep=True) \
        .groupby(['ACCOUNT NUMBER', 'SITE NUMBER']) \
        .agg({'DATE ISSUED': ['min', 'max'],
              'LICENSE TERM EXPIRATION DATE': 'max'}) \
        .reset_index(col_level=1)

    # Flatten column names into something usable
    df.columns = df.columns.to_flat_index()

    df = df.rename(columns={
        ('', 'ACCOUNT NUMBER'): "account",
        ('' , 'SITE NUMBER'): 'site',
        ('DATE ISSUED', 'min'): 'min_license_date',
        ('DATE ISSUED', 'max'): 'max_license_date',
        ('LICENSE TERM EXPIRATION DATE', 'max'): 'expiry'})

    # Extract min/max license dates into list of years_open
    df['years_open'] = pd.Series(map(lambda x, y: [z for z in range(x, y+2)],
                                     df['min_license_date'].dt.year,
                                     df['max_license_date'].dt.year))

    # make account-site id var
    # melt step below doesn't work well without merging these two cols
    df['account_site'] = df['account'].astype('str') + "-" + df['site'].astype('str')
    df = df[df.columns.tolist()[-1:] + df.columns.tolist()[:-1]]
    df = df.drop(labels=['account', 'site'], axis=1)

    # Expand list of years_open into one row for each account-site-year
    # https://mikulskibartosz.name/how-to-split-a-list-inside-a-dataframe-cell-into-rows-in-pandas-9849d8ff2401
    df = df \
        .years_open \
        .apply(pd.Series) \
        .merge(df, left_index=True, right_index=True) \
        .drop(labels=['years_open'], axis=1) \
        .melt(id_vars=['account_site', 'min_license_date', 'max_license_date',
                       'expiry'],
              value_name='YEAR') \
        .drop(labels=['variable'], axis=1) \
        .dropna() \
        .sort_values(by=['account_site', 'YEAR'])

    # Split account_site back into ACCOUNT NUMBER, SITE NUMBER
    df['ACCOUNT NUMBER'], df['SITE NUMBER'] = df['account_site'].str.split('-', 1).str
    df['ACCOUNT NUMBER'] = df['ACCOUNT NUMBER'].astype('int')
    df['SITE NUMBER'] = df['SITE NUMBER'].astype('int')

    # reorder columns
    df['YEAR'] = df['YEAR'].astype('int')
    df = df[['ACCOUNT NUMBER', 'SITE NUMBER', 'account_site', 'YEAR',
             'min_license_date', 'max_license_date', 'expiry']] \
        .sort_values(by=['ACCOUNT NUMBER', 'SITE NUMBER'])

    # Assume buffer period is last 2 years of input data
    threshold_year = input_df['DATE ISSUED'].dt.year.max() - 1
    buffer_df = input_df.loc[input_df['DATE ISSUED'].dt.year >= threshold_year]

    # Get list of account-site numbers in buffer
    buffer_ids = buffer_df['ACCOUNT NUMBER'].astype('str') \
        + '-' + buffer_df['SITE NUMBER'].astype('str')

    # Generate label
    df['not_renewed_2yrs'] = np.where(df['expiry'].dt.year < threshold_year,
        np.where(df['YEAR'] >= df['max_license_date'].dt.year + 1, 1, 0),
        np.where(df['account_site'].isin(buffer_ids),
            0,
            np.where(df['YEAR'] >= df['max_license_date'].dt.year + 1, 1, 0)
            )
        )

    # Drop unnecessary columns
    # Drop all years that we can't predict on, i.e. buffer years onwards
    df = df.drop(labels=['account_site', 'min_license_date','max_license_date',
                         'expiry'], axis=1) \
        .loc[df['YEAR'] < threshold_year] \
        .reset_index(drop=True)
    
    return df

Split this up into testing pd verses handcoded haversine:
https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas

In [ ]:
addresses = get_locations(license_data)
df = reshape_and_create_label(input_df) \
    .merge(addresses, how='left', on=['ACCOUNT NUMBER', 'SITE NUMBER'])

# Select columns, transforms lat/long in degrees to radians
df = df[['ACCOUNT NUMBER', 'SITE NUMBER', 'YEAR', 'LATITUDE', 'LONGITUDE',
         'not_renewed_2yrs']]
df['LATITUDE_rad'] = np.radians(df['LATITUDE'])
df['LONGITUDE_rad'] = np.radians(df['LONGITUDE'])
R = 6371 # circumference of the Earth in km

year_dfs = []
years = df['YEAR'].unique()

for i in sorted(df['YEAR'].unique()):
    year_df = df.loc[df['YEAR'] == i]
    fails_only = year_df.loc[year_df['not_renewed_2yrs'] == 1]

    # if no businesses failed that year, return a count of 0 for all
    if len(fails_only) == 0:
        year_df[0] = np.zeros(len(year_df)).astype('int')
        year_df = year_df \
            .reset_index(drop=True) \
            .drop(labels=['LATITUDE', 'LONGITUDE', 'LATITUDE_rad',
                          'LONGITUDE_rad', 'not_renewed_2yrs'], axis=1)
    else:
        # Get pairwise distance between all businesses that year and all
        # nonrenewals that year. Then count number of nonrenewals within
        # threshold distance (using row-wise sum) and join back on year_df
        dist_df = R * haversine_distances(
            year_df[['LATITUDE_rad', 'LONGITUDE_rad']],
            fails_only[['LATITUDE_rad', 'LONGITUDE_rad']]
        )
        dist_df = pd.DataFrame(np.where(dist_df <= 1, 1, 0).sum(axis=1))
        year_df = year_df \
            .reset_index(drop=True) \
            .join(dist_df) \
            .drop(labels=['LATITUDE', 'LONGITUDE', 'LATITUDE_rad',
                          'LONGITUDE_rad', 'not_renewed_2yrs'], axis=1)

    year_dfs.append(year_df)
# Concatenate all year-specific dfs to get counts for all business-years
# Then merge onto original df by business-year id cols
all_years_df = pd.concat(year_dfs)
results_df = input_df.merge(all_years_df,
                            how='left',
                            on=MERGE_KEYS) \
    .rename(columns={0: 'num_not_renewed_1km'}) \
    [['ACCOUNT NUMBER', 'SITE NUMBER', 'YEAR', 'num_not_renewed_1km' ]]